In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [75]:
url_inicial = 'https://www.elsevier.es/es-revista-acta-investigacion-psicologica-psychological-111-numeros-anteriores-anio-2013'
url_root = 'https://www.elsevier.es/es-revista-acta-investigacion-psicologica-psychological-111-numeros-anteriores-anio-2013'
r=requests.get(url_inicial)

In [76]:
soup = BeautifulSoup(r.text, 'html.parser')

In [77]:
box = soup.find('div', class_="sumarios decada-2013")
volumen=soup.findAll('div', class_='sumario row')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://www.elsevier.es/es-revista-acta-investigacion-psicologica-psychological-111-sumario-vol-3-num-3-S2007471913X70760',
 'https://www.elsevier.es/es-revista-acta-investigacion-psicologica-psychological-111-sumario-vol-3-num-2-S2007471913X70759',
 'https://www.elsevier.es/es-revista-acta-investigacion-psicologica-psychological-111-sumario-vol-3-num-1-S2007471913X70747']

In [78]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='items') 
    volumen1=box1.findAll('div', class_='item')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol1 = [urljoin (url_root, i) for i in vol1]
    vol2 = [urljoin (url_root, i) for i in vol2]
    vol2+=vol1

In [79]:
vol2

['https://www.elsevier.es/es-revista-acta-investigacion-psicologica-psychological-111-articulo-sources-meaning-meaning-in-life-S200747191370961X',
 'https://www.elsevier.es/es-revista-acta-investigacion-psicologica-psychological-111-articulo-validez-constructo-escala-autoridad-parental-S2007471913709621',
 'https://www.elsevier.es/es-revista-acta-investigacion-psicologica-psychological-111-articulo-propiedades-psicometricas-una-escala-evaluar-S2007471913709633',
 'https://www.elsevier.es/es-revista-acta-investigacion-psicologica-psychological-111-articulo-modelos-predictivos-homonegatividad-internalizada-estudiantes-S2007471913709645',
 'https://www.elsevier.es/es-revista-acta-investigacion-psicologica-psychological-111-articulo-efectividad-tratamiento-emdr-personas-guatemaltecas-S2007471913709657',
 'https://www.elsevier.es/es-revista-acta-investigacion-psicologica-psychological-111-articulo-las-causas-que-llevan-a-S2007471913709669',
 'https://www.elsevier.es/es-revista-acta-investig

In [80]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', class_='items') 
    volumen1=box1.findAll('div', class_='item')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [81]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina https://www.elsevier.es/es-revista-acta-investigacion-psicologica-psychological-111-numeros-anteriores-anio-2013


In [82]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

28

In [83]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [84]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Acta de Investigación Psicológica'
    a='IV. Humanidades y Ciencias de la Conducta'
    tem='Psicología'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('div', class_='elsevierItemTitulo').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='elsevierItemTextoCompletoTexto').find('p').get_text(strip=True)
        content_book['Resumen']=resu.replace("Introducción","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', id='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
        linkart = [urljoin (url_root, i) for i in linkart]
    #link articulo
    try:
        link=s_item.find('div', class_='item-anchors desktop').find('a').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
        link = [urljoin (url_root, i) for i in link]
    return content_book

In [86]:
list_scraper=list_scraper[0:28]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26


UnboundLocalError: local variable 'link' referenced before assignment

In [73]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Acta de Investigación Psicológica,IV. Humanidades y Ciencias de la Conducta,Psicología,The Effects of the Establishment of Adult Face...,"Observing the human eyes, or face, is one of t...",None,https://www.elsevier.es/es-revista-acta-invest...,https://www.elsevier.es/es-revista-acta-invest...
1,Acta de Investigación Psicológica,IV. Humanidades y Ciencias de la Conducta,Psicología,Methods for Assessing Social Validity of Behav...,"Social validity, or the extent to which consum...",None,https://www.elsevier.es/es-revista-acta-invest...,https://www.elsevier.es/es-revista-acta-invest...
2,Acta de Investigación Psicológica,IV. Humanidades y Ciencias de la Conducta,Psicología,Gathering Evidence for Distance Education1,This article describes issues that have arisen...,None,https://www.elsevier.es/es-revista-acta-invest...,https://www.elsevier.es/es-revista-acta-invest...
3,Acta de Investigación Psicológica,IV. Humanidades y Ciencias de la Conducta,Psicología,Implementation of the Good Behavior Game in Cl...,"First introduced by Barrish, Saunders, andWolf...",None,https://www.elsevier.es/es-revista-acta-invest...,https://www.elsevier.es/es-revista-acta-invest...
4,Acta de Investigación Psicológica,IV. Humanidades y Ciencias de la Conducta,Psicología,Increasing the Social Validity of Function-Bas...,"Compared to typically developing peers, childr...",None,https://www.elsevier.es/es-revista-acta-invest...,https://www.elsevier.es/es-revista-acta-invest...
5,Acta de Investigación Psicológica,IV. Humanidades y Ciencias de la Conducta,Psicología,Prácticas de Crianza Asociadas a la Reducción ...,De acuerdo con la Organización Mundial de la S...,None,https://www.elsevier.es/es-revista-acta-invest...,https://www.elsevier.es/es-revista-acta-invest...
6,Acta de Investigación Psicológica,IV. Humanidades y Ciencias de la Conducta,Psicología,Actions vs. Words: How We Can Learn Both,"In our everyday experiences, each of our sense...",None,https://www.elsevier.es/es-revista-acta-invest...,https://www.elsevier.es/es-revista-acta-invest...
7,Acta de Investigación Psicológica,IV. Humanidades y Ciencias de la Conducta,Psicología,Using Computer-Based Programmed Instruction to...,"Since 2000, there have been numerous attempts ...",None,https://www.elsevier.es/es-revista-acta-invest...,https://www.elsevier.es/es-revista-acta-invest...
8,Acta de Investigación Psicológica,IV. Humanidades y Ciencias de la Conducta,Psicología,Effects of Positive and Negative Reinforcement...,"Functional analysis (Iwata, Dorsey, Slifer, Ba...",None,https://www.elsevier.es/es-revista-acta-invest...,https://www.elsevier.es/es-revista-acta-invest...
9,Acta de Investigación Psicológica,IV. Humanidades y Ciencias de la Conducta,Psicología,"The Sciences of Learning, Instruction, and Ass...",Practices derived from the learning sciences a...,None,https://www.elsevier.es/es-revista-acta-invest...,https://www.elsevier.es/es-revista-acta-invest...


In [74]:
df_catalogo.to_csv('Revista115.05.csv', index=False)